In [ ]:
import sys
sys.path.append("..")

import random
import math
import itertools
from io import BytesIO
from pathlib import Path
from typing import Optional, Callable, List, Tuple, Iterable, Generator, Union

import PIL.Image
import PIL.ImageDraw
import plotly
import plotly.express as px
plotly.io.templates.default = "plotly_dark"
import numpy as np
import pandas as pd

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, IterableDataset, RandomSampler
import torchvision.transforms as VT
import torchvision.transforms.functional as VF
from torchvision.utils import make_grid
from IPython.display import display

from src.datasets import *
from src.util.image import *
from src.util import *
from src.algo import *
from src.models.cnn import *
from src.cluster import *

In [ ]:
def plot_histogram(labels, sorted: bool = False):
    counts, _ = np.histogram(labels, bins=int(labels.max()) + 1)
    if sorted:
        counts.sort()
    if counts.shape[0] > 1000:
        return px.line(counts)
    else:
        return px.bar(counts)

In [ ]:
!ls ../datasets/

In [ ]:
features = torch.load("../datasets/kali-uint8-64x64-CLIP.pt")#[:10000]
features.shape

In [ ]:
torch.zeros((3, 2), dtype=torch.float32)

In [ ]:
from sklearn import cluster
#clusterer = cluster.KMeans(1000, random_state=23, n_init=1, verbose=2)
clusterer = cluster.MiniBatchKMeans(1000, batch_size=1024, random_state=23, n_init=1, verbose=2)
dl = DataLoader(features, batch_size=1024)
for batch in tqdm(dl):
    clusterer.partial_fit(batch)

In [ ]:
labels = clusterer.predict(features)
plot_histogram(labels, sorted=True)

In [ ]:
clusterer = AutoCluster(
    distance_threshold=.01,
    max_n_clusters=100,
    verbose=True,
)
labels = clusterer.fit_transform(np.random.rand(10000, 512))
plot_histogram(labels, sorted=True)

In [ ]:
labels